In [68]:
from IPython.display import display
import pandas as pd

In [69]:
caminho_arquivo = "/home/danielbarbosa/seget/notebooks/dados_abertos_cnpj/dados_brutos/estabelecimentos9.csv"

caminho_saida = "/home/danielbarbosa/seget/notebooks/dados_abertos_cnpj/dados_tratados/estabelecimentos9_tratado.csv"

chunk_size = 100_000

novos_nomes_colunas = [
    "cnpj_basico", "cnpj_ordem", "cnpj_dv", "identificador_matriz_filial", "nome_fantasia",
    "situacao_cadastral", "data_situacao_cadastral", "motivo_situacao_cadastral",
    "nome_cidade_exterior", "pais", "data_inicio_atividade", "cnae_fiscal_principal",
    "cnae_fiscal_secundaria", "tipo_logradouro", "logradouro", "numero", "complemento",
    "bairro", "cep", "uf", "municipio", "ddd1", "telefone1", "ddd2", "telefone2", "ddd_fax",
    "fax", "correio_eletronico", "situacao_especial", "data_situacao_especial"
]

In [70]:
# Lê os primeiros 3 chunks sem header e atribui os nomes corretos às colunas
chunks = pd.read_csv(
    caminho_arquivo, 
    sep=';', 
    encoding='ISO-8859-1', 
    header=None,  # Evita que o pandas use a primeira linha como cabeçalho
    names=novos_nomes_colunas,  # Define os nomes corretos das colunas
    chunksize=chunk_size,
    low_memory=False
)

# Concatenar os 3 primeiros chunks em um único DataFrame
df_estabelecimentos = pd.concat([next(chunks) for _ in range(3)], ignore_index=True)

# Filtrando apenas os estabelecimentos de Maricá (código do município = 5853)
df_marica = df_estabelecimentos[df_estabelecimentos["municipio"] == 5853].copy()

pd.set_option('display.width', 1000)  # Define uma largura maior para o display
pd.set_option('display.max_columns', None)  # Mostra todas as colunas
pd.set_option('display.max_colwidth', None)  # Evita truncar valores de strings

In [71]:
# Tratamento de valores NaN
df_marica['nome_fantasia'] = df_marica['nome_fantasia'].fillna('Não informado')
df_marica['nome_cidade_exterior'] = df_marica['nome_cidade_exterior'].fillna('Não informado')
df_marica['pais'] = df_marica['pais'].fillna('Não informado')
df_marica['cnae_fiscal_secundaria'] = df_marica['cnae_fiscal_secundaria'].fillna('Não informado')
df_marica['complemento'] = df_marica['complemento'].fillna('Não informado')
df_marica['correio_eletronico'] = df_marica['correio_eletronico'].fillna('Não informado')
df_marica['situacao_especial'] = df_marica['situacao_especial'].fillna('Não informado')

df_marica['telefone1'] = df_marica['telefone1'].fillna('Não informado')
df_marica['telefone2'] = df_marica['telefone2'].fillna('Não informado')
df_marica['fax'] = df_marica['fax'].fillna('Não informado')

# Função para tratar a coluna de DDD, removendo .0 e mantendo "Não informado"
def tratar_ddd(valor):
    if pd.isna(valor):  # Se for NaN, retorna "Não informado"
        return "Não informado"
    return str(int(valor)) if isinstance(valor, (int, float)) else str(valor)

# Aplicar a função nas colunas de DDD
df_marica['ddd1'] = df_marica['ddd1'].apply(tratar_ddd)
df_marica['ddd2'] = df_marica['ddd2'].apply(tratar_ddd)
df_marica['ddd_fax'] = df_marica['ddd_fax'].apply(tratar_ddd)

# Converter a coluna para string, caso ainda não esteja no formato adequado
df_marica['data_situacao_cadastral'] = df_marica['data_situacao_cadastral'].astype(str)

# Converter a coluna para o formato de data correto (YYYY-MM-DD)
df_marica['data_situacao_cadastral'] = pd.to_datetime(df_marica['data_situacao_cadastral'], format='%Y%m%d', errors='coerce')

# Converter a coluna para string, caso ainda não esteja no formato adequado
df_marica['data_inicio_atividade'] = df_marica['data_inicio_atividade'].astype(str)

# Converter a coluna para o formato de data correto (YYYY-MM-DD)
df_marica['data_inicio_atividade'] = pd.to_datetime(df_marica['data_inicio_atividade'], format='%Y%m%d', errors='coerce')

# Converter para string, garantindo que os valores possam ser tratados corretamente
df_marica['data_situacao_especial'] = df_marica['data_situacao_especial'].astype(str)

# Substituir valores vazios ou "NaN" por "Não informado"
df_marica['data_situacao_especial'] = df_marica['data_situacao_especial'].replace(['nan', 'NaN', 'Não informado'], pd.NA)

# Converter para formato de data corretamente
df_marica['data_situacao_especial'] = pd.to_datetime(df_marica['data_situacao_especial'], format='%Y%m%d', errors='coerce')

# Substituir valores NaT por "Não informado" e formatar as datas existentes
df_marica['data_situacao_especial'] = df_marica['data_situacao_especial'].apply(
    lambda x: 'Não informado' if pd.isna(x) else x.strftime('%Y-%m-%d')
)

# Remover colunas desnecessárias
colunas_para_remover = ["motivo_situacao_cadastral", "nome_cidade_exterior", "pais"]
df_marica.drop(columns=colunas_para_remover, inplace=True, errors='ignore')

# Salvar o DataFrame no diretório especificado
df_marica.to_csv(caminho_saida, sep=';', index=False, encoding="utf-8-sig")

# display(df_marica.head(3))
